# 钱包

钱包是用来存放用户私钥。

## 不确定性钱包
当用户需要使用新地址时，钱包将随机生成私钥并计算地址，私钥生成的方式无规律可循。

In [1]:
from simchain import Wallet

# 创建一个钱包
w = Wallet()

# 随机生成10对密钥
for _ in range(10):
    w.generate_keys()
w.nok

10

2019-04-29 09:36:21,080 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# 访问最后一对密钥中私钥的字节串编码
w.keys[-1].sk.to_bytes()

b"\x05\x8eE\xb0\x8a'\xedKEd\x87\xc1(7+\x01d\xa8n\xbb\x10\xd7'5\xf9\x8d\xb9\xc4\xcf\x88\xf15"

In [3]:
# 访问倒数第二对密钥中私钥的字节串编码
w.keys[8].sk.to_bytes()

b"\x95\x1c\x0bhG\xc8\x83'o\x80\xb8;\xd0\xbb\xc5\xfb>\x91\xfd\x15p\x06\x1b\x9e@\x94IQ\x95\xa2\xc7\xfd"

## 分层确定性钱包（Hierachical Deterministic Wallets）

### 种子
也称为“种子”钱包，“种子”通过密码学上安全的伪随机数生成器，生成128、256或512位的随机数。然后通过HMAC-SHA512运算得到一个512位的输出，输出的左边256位为主私钥，右边256位为主链码，主公钥和地址通过主私钥生成。

In [4]:
import os
import hashlib
import hmac
from simchain import SigningKey

# 生成一个随机种子，256位
master_seed = os.urandom(32)
master_seed

b'\xb6\xba\xe2\xbbg\x8b\x84\xd6\xf9f\x83\xa3W\xfe\xaf\x04g6L\xf3\x9e\xc6e\x9f\x8do\xd4\xd2\xa1\xfc\x89O'

In [5]:
# 使用HMAC-SHA512运算得到512位输出
deriv = hmac.new(key=b'Simchain seed', msg=master_seed, digestmod=hashlib.sha512).digest()

# 取输出的左边256位生成主私钥
master_privkey_str = deriv[:32]
master_privkey = SigningKey.from_bytes(master_privkey_str)
master_privkey.to_bytes()

b'\x85\xbb\xc9\xa0\x9c1\x89\x9e6w\xf5=}\xfd\x7f\xd3zQ\x01+\xf6\x95\xcc\x85\xd6\xff\x87_#\xa2\x87f'

In [6]:
# 由主私钥生成主公钥
master_pubkey = master_privkey.get_verifying_key()
master_privkey.to_bytes()

b'\x85\xbb\xc9\xa0\x9c1\x89\x9e6w\xf5=}\xfd\x7f\xd3zQ\x01+\xf6\x95\xcc\x85\xd6\xff\x87_#\xa2\x87f'

In [7]:
from simchain.ecc import convert_pubkey_to_addr
# 由主公钥生成主地址
convert_pubkey_to_addr(master_privkey.to_bytes())

'1LEoXuo2A9moCjJ6DgoErUJd6MX3itf7da'

### 衍生密钥
以父私钥、父链码，以及子密钥索引号作为HMAC-SHA512运算的输入，得到512位输出的左边256位作为子私钥，右边256位作为子链码，子公钥和子地址通过子私钥生成。

In [8]:
# 获得主链码
master_chain = deriv[32:]

from simchain.ecc import number_to_bytes

# 衍生主私钥的索引为1的子密钥
idx = 1
# 将整数转换为字节串
idx_str = number_to_bytes(idx, 4)
idx_str

b'\x00\x00\x00\x01'

In [9]:
# 将主私钥与索引组合
message = master_privkey_str + idx_str
# 将主链码作为钥匙，主私钥和索引组合作为消息进行HMAC-SHA512运算
deriv_child = hmac.new(key=master_chain, msg=message, digestmod=hashlib.sha512).digest()
# 取输出的左256位作为子私钥
child_privkey_str = deriv_child[:32]
child_privkey_str

b'\xf0!\x86\xdd7>\xef@\xdbv\xba\x07\xfb#\x03\x1b\xceW\xcf\x9c8?\xb3\xd4FI\x9d\xc6\xc4\xc0\x0c+'

In [10]:
# 生成子私钥
child_privkey = SigningKey.from_bytes(child_privkey_str)
child_privkey.to_bytes()

b'\xf0!\x86\xdd7>\xef@\xdbv\xba\x07\xfb#\x03\x1b\xceW\xcf\x9c8?\xb3\xd4FI\x9d\xc6\xc4\xc0\x0c+'

In [11]:
# 生成子公钥
child_pubkey = child_privkey.get_verifying_key()
child_pubkey.to_bytes()

b'1\xf5\x03b\x08j\xe1K\xd5\xc3\xa0Z\x84\xb8\x9c:\x9b\x86\xb1\x8b\xde\xdc\x18\xd5w\x80\x1bD[M8P,\xc6\xca\xa5\x12\x07\x7f1V\x98\x15\x05\xd9\x10J\xbd\x12\xd6\xd2\xae0\x164\x99\xa8\xed\xfd\x04\x15O\xf6?'

In [12]:
# 生成子地址
convert_pubkey_to_addr(child_pubkey.to_bytes())

'1Jfpb3D4bzxdUriK16WijsGuUZT8qkwg2z'

### 生成分层确定性钱包

In [13]:
from simchain.hdwallet import Keys
import os

In [14]:
# 生成随机种子
seed = os.urandom(32)
# 通过种子生成主密钥
master_keys = Keys.from_master_seed(seed)
# 访问主密钥私钥的字节串编码
master_keys.sk.to_bytes()

b'\x12\xa6\xe7\xb5Q\xb9\x1c\x8f\xc1:\x99\x0e-\xa1+\xcb\xcf\x02\n\xde\x96\xb1y\x0bes\xba\x87\x97<c\x9c'

In [15]:
# 访问主密钥公钥的字节串编码
master_keys.pk.to_bytes()

b"Y\x98T\xd1\xf9\xc4\x82+_\xfdZ \xc3\x11(\xd2^{ZU@\xe7x\xed@\x95\x17 V\x16S'\xfak\x8a\xd7{\xe4\xf7p\xdc\xd5\t\x03\x95\xc2\xee\xd8{\xb2}\xa5P>c\x16<\xca\xe3\x8d}C!\x13"

In [16]:
# 主密钥的深度为0
master_keys.depth

0

In [17]:
# 返回为None，表示没有父亲
master_keys.child_index

In [18]:
# 主密钥衍生成2个字密钥，索引为0和1
child0 = master_keys.child(0)
child1 = master_keys.child(1)
# 子密钥的深度都为1
assert child0.depth == child1.depth

In [19]:
child0.child_index

0

In [20]:
child1.child_index

1

In [21]:
child0.sk.to_bytes()

b'\x1aQ\xebd\xe1;s4\x83=\xc7.}t\xc9\x83\xa4\x10|\xc4x\xea\x89~\xc1\x13x\xa4\xb6\xfb\x9d\xc4'

In [22]:
child0.pk.to_bytes()

b'~\r\xadw\xb9\x96\x97\n\xf6T\xffA-o\xe7M;\xca:+\x9aB5Wb\r7\xe1E\xeeJ\t\x02.{\xe2\xb7\xc9U\xd1\x91e\xbe\x1fUG\xee\x80I\xbf\x14\x04\xc4I\xd3\xb0\xe5\xb9%\x89\x8b\x80\xec\x08'

In [23]:
# 子密钥链码
child0.chain

b'\xa9\x998\xdb\xf8\xad\xa2\xf3%e}V\xa0\xc4\xaf\xe1\xf0\\0?\x85s\x08\xbcd\xec\xea){\xac!\xe9'

In [24]:
# 生成孙密钥，索引分别为2和4
grandson2 = child0.child(2)
grandson4 = child1.child(4)
# 孙密钥深度为2
assert grandson2.depth == grandson4.depth == 2

In [25]:
grandson2.child_index

2

In [26]:
grandson4.child_index

4

### 将“种子”转换为可逆的助记符 

In [27]:
from simchain.mnemonics import Mnemonics
# 生成种子
seed = os.urandom(32)
seed

b"\x1e\x15\xbdq\xeb\x13eY\n\xe5\xd3\xee\xc7\xaf\xccB\x1f\xf3\xfc\x9fW\xbc\x18'\x94i\xaa\xc6\xec+~u"

In [28]:
# 生成助记词对象
m = Mnemonics(seed)
# 中文助记符
m.chinese

['颓',
 '徒',
 '烷',
 '壮',
 '魁',
 '剩',
 '助',
 '诗',
 '城',
 '贬',
 '渡',
 '龟',
 '惧',
 '菜',
 '牡',
 '飞',
 '茂',
 '邻',
 '柱',
 '世',
 '等',
 '馐',
 '启',
 '气']

In [29]:
# 英文助记符
m.english

['cheat',
 'due',
 'prefer',
 'reason',
 'stun',
 'harsh',
 'begin',
 'gaze',
 'offer',
 'bloom',
 'perfection',
 'sharp',
 'rope',
 'excuse',
 'drove',
 'better',
 'sanctuary',
 'floor',
 'grab',
 'whole',
 'cheer',
 'dirty',
 'piece',
 'happy']

In [30]:
# 由中文助记词得到种子
seed1 = Mnemonics.decode_from_chinese(m.chinese)
seed1

b"\x1e\x15\xbdq\xeb\x13eY\n\xe5\xd3\xee\xc7\xaf\xccB\x1f\xf3\xfc\x9fW\xbc\x18'\x94i\xaa\xc6\xec+~u"

In [31]:
# 由英文助记词得到种子
seed2 = Mnemonics.decode_from_english(m.english)
seed2

b"\x1e\x15\xbdq\xeb\x13eY\n\xe5\xd3\xee\xc7\xaf\xccB\x1f\xf3\xfc\x9fW\xbc\x18'\x94i\xaa\xc6\xec+~u"